# Analyze results
Aug 28, 2020

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *


In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [7]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [8]:
img_size=128
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [9]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]

dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20201212_190922_cgan_long_run_model5', '20201210_073…

In [10]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201213_112641_lambda0.1


In [11]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'128':raw_data_dir+'128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
                 '512':raw_data_dir+'512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'}

## Plot Losses

In [12]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [13]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
62491,62491.0,19.0,0.112923,0.113961,0.226884,7.977307,16.070469,8.093163,-3.059265,8.302679,-2.878974,3.890522,-6.518106,-7.976525,0.134038
62492,62492.0,19.0,0.060213,0.104026,0.164239,4.975972,13.301744,8.325771,-3.017022,8.548458,-2.923136,4.023936,-5.099908,-4.960206,0.135298
62493,62493.0,19.0,0.088903,0.189684,0.278588,10.145069,19.106939,8.961870,-3.061364,8.257876,-3.000869,4.685781,-2.074656,-10.144995,0.133840
62494,62494.0,19.0,0.281371,0.132737,0.414108,7.477383,15.394178,7.916796,-2.974538,8.058998,-2.755051,2.527019,-7.894518,-7.476290,0.132238
62495,62495.0,19.0,0.085730,0.074685,0.160416,4.618678,13.192236,8.573558,-2.651344,8.557017,-2.538425,5.222563,-5.104132,-4.601155,0.132866
62496,62496.0,19.0,0.145172,0.406900,0.552072,17.031229,25.644958,8.613730,-2.722464,8.654496,-2.774181,6.475261,-1.080797,-17.031229,0.134400
62497,62497.0,19.0,0.835641,0.234410,1.070051,11.643472,20.218525,8.575053,-2.903120,8.661022,-2.888177,0.329494,-14.079641,-11.643441,0.133767
62498,62498.0,19.0,0.080577,0.114889,0.195466,6.684882,14.965102,8.280220,-2.959425,8.444351,-2.810938,7.248598,-8.609193,-6.682144,0.132222
62499,62499.0,19.0,0.156237,0.192336,0.348573,6.395264,14.792780,8.397516,-3.027807,8.434612,-2.960248,11.364698,-3.069343,-6.391979,0.133702
62500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.121996,-3.048619,NaN,NaN,NaN,NaN


In [14]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [15]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-3.204855613708496


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
60386,60386.0,19.0,0.134438,0.133552,0.267991,6.617105,15.117842,8.500736,-3.368162,7.916311,-3.376441,4.428882,-3.150840,-6.614693,0.133089
62306,62306.0,19.0,0.088351,0.111375,0.199726,6.836541,15.688099,8.851559,-3.319578,8.328160,-3.363294,4.775371,-5.232479,-6.834782,0.128724
62302,62302.0,19.0,0.127741,0.221549,0.349290,10.267921,18.587042,8.319120,-3.255391,8.199100,-3.362725,5.138275,-2.234577,-10.267853,0.130828
60387,60387.0,19.0,0.161095,0.139964,0.301059,6.328604,14.325046,7.996442,-3.320465,7.915676,-3.356003,3.595526,-3.837674,-6.324641,0.183759
62308,62308.0,19.0,0.099455,0.080382,0.179837,5.966459,14.346601,8.380143,-3.274680,7.973623,-3.355870,4.058745,-3.194024,-5.959981,0.129037
57960,57960.0,18.0,0.181259,0.193768,0.375028,11.632711,19.589321,7.956609,-3.237112,8.011706,-3.346698,3.795450,-11.336142,-11.632689,0.142015
60378,60378.0,19.0,0.093237,0.519246,0.612483,16.077370,24.455708,8.378339,-3.191162,8.277215,-3.344228,7.411614,-0.999865,-16.077370,0.130567
47697,47697.0,15.0,0.304206,0.049495,0.353702,3.596727,11.682287,8.085560,-3.271202,8.314813,-3.344067,1.798942,-4.918662,-3.525007,0.132818
60338,60338.0,19.0,0.092848,0.140337,0.233185,6.992467,15.106213,8.113745,-3.102453,8.153993,-3.342332,4.066267,-3.058758,-6.990665,0.133460
54901,54901.0,17.0,0.116664,0.086848,0.203512,6.066790,14.689734,8.622945,-2.846997,8.028460,-3.337610,4.539694,-4.649626,-6.061409,0.135220


In [16]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
display(df_metrics.sort_values(by=['spec_chi']).head(8))

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
60386,60386.0,19.0,0.134438,0.133552,0.267991,6.617105,15.117842,8.500736,-3.368162,7.916311,-3.376441,4.428882,-3.150840,-6.614693,0.133089
62306,62306.0,19.0,0.088351,0.111375,0.199726,6.836541,15.688099,8.851559,-3.319578,8.328160,-3.363294,4.775371,-5.232479,-6.834782,0.128724
62302,62302.0,19.0,0.127741,0.221549,0.349290,10.267921,18.587042,8.319120,-3.255391,8.199100,-3.362725,5.138275,-2.234577,-10.267853,0.130828
60387,60387.0,19.0,0.161095,0.139964,0.301059,6.328604,14.325046,7.996442,-3.320465,7.915676,-3.356003,3.595526,-3.837674,-6.324641,0.183759
62308,62308.0,19.0,0.099455,0.080382,0.179837,5.966459,14.346601,8.380143,-3.274680,7.973623,-3.355870,4.058745,-3.194024,-5.959981,0.129037
57960,57960.0,18.0,0.181259,0.193768,0.375028,11.632711,19.589321,7.956609,-3.237112,8.011706,-3.346698,3.795450,-11.336142,-11.632689,0.142015
60378,60378.0,19.0,0.093237,0.519246,0.612483,16.077370,24.455708,8.378339,-3.191162,8.277215,-3.344228,7.411614,-0.999865,-16.077370,0.130567
47697,47697.0,15.0,0.304206,0.049495,0.353702,3.596727,11.682287,8.085560,-3.271202,8.314813,-3.344067,1.798942,-4.918662,-3.525007,0.132818


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,D(x),D_G_z1,D_G_z2,time
27334,27334.0,8.0,0.071428,0.173868,0.245296,4.346965,12.755826,8.408861,-0.793136,7.472477,-0.854610,4.233376,-2.771155,-4.327902,0.127720
17579,17579.0,5.0,0.354414,0.211256,0.565670,2.527702,10.718909,8.191208,-1.972101,7.473513,-2.088990,1.428270,-2.114132,-2.400271,0.133461
28263,28263.0,9.0,0.155216,0.102437,0.257653,3.461741,11.864871,8.403131,-0.823498,7.526837,-0.775764,2.774772,-3.233370,-3.400209,0.135036
31672,31672.0,10.0,0.163531,1.350837,1.514368,11.438201,19.676743,8.238542,-1.399827,7.534982,-1.415339,8.975668,0.822421,-11.438184,0.130889
26672,26672.0,8.0,0.126929,2.175810,2.302739,10.542264,18.384790,7.842526,-0.974653,7.535458,-0.965391,7.678528,1.970892,-10.542221,0.132915
26371,26371.0,8.0,0.094699,0.365968,0.460667,5.469049,13.544851,8.075802,-0.859944,7.543489,-0.926054,4.378223,-1.324427,-5.463110,0.133286
27534,27534.0,8.0,0.324574,0.090877,0.415451,2.374754,10.766867,8.392113,-0.603001,7.551349,-0.750657,1.636324,-3.795386,-2.222131,0.129592
31539,31539.0,10.0,0.107437,0.104423,0.211861,8.058577,16.136608,8.078031,-1.117297,7.560135,-1.125052,5.291790,-4.869569,-8.057575,0.130004


## Read stored chi-squares for images

In [17]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting


col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [18]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [19]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [20]:
df_sliced=w.result
# df_sliced

In [21]:
best_step=[]
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_1',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.8,sort_col='chi_1c',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_spec1',head=2,display_flag=False).step.values)
print(best_step)
best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 0.08297711864228774, 'chi_spec1': 0.9375246794024896}
[0, 19]
{'chi_1': 0.2196426834685159, 'chi_spec1': 2.163042052374275}
[0, 19]
{'chi_1': 0.08297711864228774, 'chi_spec1': 0.9375246794024896}
[array([12580, 11890]), array([32650, 33240]), array([13360, 14070])]
[11890 12580 13360 14070 32650 33240]


In [18]:
# best_step=[58800]
# best_step=np.arange(32800,32900,10).astype(int)

In [22]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(6, 22)
[(3, 11890), (4, 12580), (4, 13360), (4, 14070), (10, 32650), (10, 33240)]


In [23]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
1188,3-11890,0.005540,0.411640,25.638957,0.001095,0.002714,0.001731,0.000855,12.585975,29.252951,3,11890
1257,4-12580,0.005435,0.366946,30.624920,0.001008,0.002032,0.002395,0.000250,9.988509,22.640762,4,12580
1335,4-13360,0.026239,0.079673,27.586688,0.014332,0.003604,0.008303,0.011215,11.304398,7.060664,4,13360
1406,4-14070,0.060474,0.103393,25.645682,0.015360,0.004154,0.040959,0.004707,11.220060,7.777181,4,14070
3264,10-32650,0.051867,0.323149,28.938008,0.047224,0.003542,0.001101,0.027628,12.547726,28.522647,10,32650
3323,10-33240,0.037621,0.923542,33.149135,0.020990,0.015483,0.001148,0.008717,13.003406,71.099732,10,33240


### Plots 2: stored data

In [24]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

(1000, 128, 128)


/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/modules_image_analysis.py:85: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  hist_arr=np.array([np.histogram(arr.flatten(), bins=bins, range=(llim,ulim), density=norm) for arr in img_arr]) ## range is important


In [25]:

def f_plot_hist_spec(df,dict_bkg,plot_type):

    assert plot_type in ['hist','spec','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='grid':
            images=np.load(row.fname)[:,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
        
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
    

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_best,dict_val,'grid')



In [26]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)
    
interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid']))

interactive(children=(SelectMultiple(description='labels_list', options=('3-11890', '4-12580', '4-13360', '4-1…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [31]:
ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(ip_fname,mmap_mode='r')[:1000,0,:,:]
print(s_val.shape)
f_plot_grid(s_val[100:118],cols=6,fig_size=(6,3))



(1000, 128, 128)
3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …